## 1 - API / Data-Collection

In [2]:
import requests
import pandas as pd
import uuid
from datetime import datetime

class ThurgauAPIClient:
    """Client to fetch and clean Open Government Data from Thurgau."""

    def __init__(self, base_url: str = "https://data.tg.ch/api/records/1.0/search/"):
        self.base_url = base_url

    def fetch_data(self, dataset: str, max_records: int = 1000) -> pd.DataFrame:
        """Fetches data from the Thurgau - Open Government Data API.

        Args:
            dataset (str): Unique dataset ID from the webpage's API description.
            max_records (int, optional): Max rows to fetch. Defaults to 1000.

        Returns:
            pd.DataFrame: Table with requested data.
        """
        
        params = {"dataset": dataset, "rows": max_records}
        response = requests.get(self.base_url, params=params)
        response.raise_for_status()
        data = response.json()
        records = [r["fields"] for r in data.get("records", [])]
        df = pd.DataFrame(records)
        print(f"Es wurden {len(df)} Datensätze vom Dataset '{dataset}' geladen.")
        return df

class DataCleaner:
    """Utility class for cleaning and enriching datasets with additional columns."""

    @staticmethod
    def add_uuid(df: pd.DataFrame, uuid_column: str = "uuid") -> pd.DataFrame:
        """Add a unique UUID to each row."""
        df[uuid_column] = [str(uuid.uuid4()) for _ in range(len(df))]
        return df

    @staticmethod
    def add_timestamp(df: pd.DataFrame, ts_column: str = "loaded_at") -> pd.DataFrame:
        """Add a timestamp to each row."""
        df[ts_column] = datetime.now().strftime("%Y-%m-%d")
        return df


# Execution
client = ThurgauAPIClient()
df_c02 = client.fetch_data(dataset="div-energie-8")

cleaner = DataCleaner()
df_c02 = cleaner.add_uuid(df_c02)
df_c02 = cleaner.add_timestamp(df_c02)

df_c02


Es wurden 720 Datensätze vom Dataset 'div-energie-8' geladen.


,bfs_nr_gemeinde,erdoelbrennstoffe,gemeinde_name,jahr,energiebezugsflaeche,total,einwohner,erdgas,andere,uuid,loaded_at
0,4881,1686.933,Amlikon-Bissegg,2015,113791,1686.933,1320,NaN,NaN,f1992d16-9e8e-47db-b041-29b612c3279c,2025-07-31
1,4921,2217.241,Bussnang,2015,187360,3151.487,2262,932.375,1.871,14edad15-e067-484b-ace1-cdce27394aab,2025-07-31
2,4751,2836.259,Rickenbach (TG),2015,227233,4585.831,2766,1745.491,4.081,012128bd-81d7-49b7-832e-cb47d453258f,2025-07-31
3,4756,867.692,Schönholzerswilen,2015,72638,867.692,804,NaN,NaN,3fd1a913-9812-4dae-a54f-6e7d3a765265,2025-07-31
4,4696,3593.640,Tägerwilen,2015,348780,6037.877,4377,2444.237,NaN,e19449d4-c3a6-4162-ab36-25499557ae5c,2025-07-31
...,...,...,...,...,...,...,...,...,...,...,...
715,4806,1092.829,Eschenz,2023,168717,2234.899,1893,1142.070,NaN,f1cf04a4-0c76-4d2f-a73c-2bd5ef5910d3,2025-07-31
716,4724,2305.944,Eschlikon,2023,430312,5319.303,4864,3013.359,NaN,e2998d04-cd72-4af5-b39a-9628be80aeb3,2025-07-31
717,4741,1227.117,Lommis,2023,103746,1227.117,1267,NaN,NaN,b449ee7f-b5ae-4cc7-98b1-e6bea0ea04dd,2025-07-31
718,4441,1192.134,Salmsach,2023,116236,1862.043,1578,669.910,NaN,23f53079-78bd-4f81-8a6f-f6e0a2959f7b,2025-07-31


In [3]:
# Export locally
df_c02.to_csv("20250718_export.csv", sep=";", index=None)

In [15]:
len(df_c02["gemeinde_name"].unique())


80

In [8]:
df_test = df_c02[df_c02["gemeinde_name"] == "Mammern"].sort_values(by=["jahr"])
df_test["C02/qm"] = df_test["total"] / df_test["energiebezugsflaeche"]
df_test[["gemeinde_name", "jahr", "energiebezugsflaeche", "total", "C02/qm"]]

,gemeinde_name,jahr,energiebezugsflaeche,total,C02/qm
495,Mammern,2015,66549,2118.685,0.031836
278,Mammern,2016,65230,2122.518,0.032539
426,Mammern,2017,65254,2145.186,0.032874
675,Mammern,2018,65660,2128.088,0.032411
686,Mammern,2019,69739,2220.267,0.031837
556,Mammern,2020,71164,2172.375,0.030526
83,Mammern,2021,72700,2434.422,0.033486
398,Mammern,2022,314455,2032.577,0.006464
348,Mammern,2023,314976,1963.324,0.006233


array(['Mammern'], dtype=object)

## 2 - Analysis

In [9]:
import pandas as pd

class AnalyseC02Data:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.tables = {}  # Speicherung der dfs

        # Data Cleaning
        self.df['andere'] = self.df['andere'].fillna(0)
        #Renaming
        self.df = self.df.rename(columns={"total": "c02_emissions"})

        # Analysen
        self._prepare_kpis_pro_jahr()
        self._prepare_lookerstudio_table()
        self._prepare_energiemix()
        self._prepare_gemeindeentwicklung()
        self._prepare_gemeindedetails()
        self._prepare_entwicklung_einwohner_vs_C02()

    def _prepare_kpis_pro_jahr(self):
        df = self.df.groupby('jahr').agg({
            'c02_emissions': 'sum',
            'einwohner': 'sum'
        }).reset_index()

        df['energy_per_inhabitant'] = df['c02_emissions'] / df['einwohner']
        df['growth_total_energy_pct'] = df['c02_emissions'].pct_change() * 100
        df['growth_energy_per_inhabitant_pct'] = df['energy_per_inhabitant'].pct_change() * 100

        # Cast datatypes
        df["jahr"] = pd.to_datetime(df["jahr"].astype(str) + "-01-01")

        self.tables['kpi_pro_jahr'] = df

    def _prepare_lookerstudio_table(self):
        df = self.df.copy()

        # Optional: Datum im Format "jahr" als echtes Datum (z. B. 01.01.2023)
        df["jahr"] = pd.to_datetime(df["jahr"].astype(str) + "-01-01")

        # Kein groupby — keine Aggregation!
        df = df[["jahr", "gemeinde_name", "einwohner", "c02_emissions", "energiebezugsflaeche"]].copy()

        self.tables["lookerstudio_base"] = df


    def _prepare_energiemix(self):
        df = self.df.copy()
        # Summieren der Energiearten pro jahr
        df_grouped = df.groupby("jahr")[["erdoelbrennstoffe", "erdgas", "andere"]].sum().reset_index()

        # Umwandeln in Long-Format
        df = df_grouped.melt(
            id_vars="jahr",
            value_vars=["erdoelbrennstoffe", "erdgas", "andere"],
            var_name="energietraeger",
            value_name="menge"
        )

        df["anteil_prozent"] = df.groupby("jahr")["menge"].transform(lambda x: round(x / x.sum() * 100, 2))

        # Cast datatypes
        df["jahr"] = pd.to_datetime(df["jahr"].astype(str) + "-01-01")

        self.tables['energiemix_pro_jahr'] = df


    def _prepare_gemeindeentwicklung(self):
        df = self.df.copy()
        df['energy_per_inhabitant'] = df['c02_emissions'] / df['einwohner']

        df = df[['bfs_nr_gemeinde', 'gemeinde_name', 'jahr', 'c02_emissions', 'energy_per_inhabitant']]

        # Cast datatypes
        df["jahr"] = pd.to_datetime(df["jahr"].astype(str) + "-01-01")

        self.tables['gemeindeentwicklung'] = df

    def _prepare_gemeindedetails(self):
        df = self.df.copy()
        df['energy_per_inhabitant'] = df['c02_emissions'] / df['einwohner']

        df = df[[
            'bfs_nr_gemeinde', 'gemeinde_name', 'jahr',
            'c02_emissions', 'erdoelbrennstoffe', 'erdgas', 'andere',
            'einwohner', 'energiebezugsflaeche', 'energy_per_inhabitant'
        ]]

        # Cast datatypes
        df["jahr"] = pd.to_datetime(df["jahr"].astype(str) + "-01-01")

        self.tables['gemeindedetails'] = df

    def _prepare_entwicklung_einwohner_vs_C02(self):
        df = self.df.copy()
        df = df[["jahr", "gemeinde_name", "einwohner", "c02_emissions", "energiebezugsflaeche"]].copy()

        df["C02_pro_qm"] = df["c02_emissions"] / df["energiebezugsflaeche"] * 1000
        df["C02_pro_einwohner"] = df["c02_emissions"] / df["einwohner"] * 1000

        # Cast datatypes
        df["jahr"] = pd.to_datetime(df["jahr"].astype(str) + "-01-01")

        self.tables["einwohner_vs_c02"] = df


    def get_table(self, name: str) -> pd.DataFrame:
        return self.tables.get(name)

    def get_all_tables(self) -> dict:
        return self.tables

## 3 - BigQuery Upload

In [10]:
from google.oauth2 import service_account
from pandas_gbq import to_gbq
import os

class BigQueryUploader:
    def __init__(self, project_id: str, credentials_path: str):
        self.project_id = project_id
        self.credentials_path = credentials_path
        self.credentials = service_account.Credentials.from_service_account_file(credentials_path)

    def upload_tables(self, tables: dict, if_exists: str = "replace"):
        """
        tables: dict im Format {
            "name1": {
                "dataframe": df1,
                "dataset": "mein_dataset",
                "table": "meine_tabelle"
            },
            ...
        }
        """
        for name, config in tables.items():
            df = config["dataframe"]
            dataset = config["dataset"]
            table = config["table"]
            full_table_name = f"{dataset}.{table}"

            print(f"⬆️ Lade {name} hoch nach: {full_table_name}...")

            try:
                to_gbq(
                    dataframe=df,
                    destination_table=full_table_name,
                    project_id=self.project_id,
                    credentials=self.credentials,
                    if_exists=if_exists
                )
                print(f"✅ {name} erfolgreich hochgeladen.\n")
            except Exception as e:
                print(f"❌ Fehler beim Hochladen von {name}: {e}\n")

In [11]:
# Analyse und Uploadinfos vorbereiten
analyse = AnalyseC02Data(df_c02)
upload_dict = {}
for name, df in analyse.get_all_tables().items():
    upload_dict[name] = {
        "dataframe": df,
        "dataset": "energie_daten",
        "table": name  # Tabelle im BQ trägt denselben Namen wie die Analyse
    }

#Upload
uploader = BigQueryUploader(
    project_id="c02-tg",
    credentials_path=os.path.join(os.pardir, "secrets", "bigquery-service-account-c02-tg.json")
)
uploader.upload_tables(upload_dict)

⬆️ Lade kpi_pro_jahr hoch nach: energie_daten.kpi_pro_jahr...
✅ kpi_pro_jahr erfolgreich hochgeladen.

⬆️ Lade lookerstudio_base hoch nach: energie_daten.lookerstudio_base...
✅ lookerstudio_base erfolgreich hochgeladen.

⬆️ Lade energiemix_pro_jahr hoch nach: energie_daten.energiemix_pro_jahr...
✅ energiemix_pro_jahr erfolgreich hochgeladen.

⬆️ Lade gemeindeentwicklung hoch nach: energie_daten.gemeindeentwicklung...
✅ gemeindeentwicklung erfolgreich hochgeladen.

⬆️ Lade gemeindedetails hoch nach: energie_daten.gemeindedetails...
✅ gemeindedetails erfolgreich hochgeladen.

⬆️ Lade einwohner_vs_c02 hoch nach: energie_daten.einwohner_vs_c02...
✅ einwohner_vs_c02 erfolgreich hochgeladen.

